# ML.py, MLpy, MLPy!
<p style="text-align: center;">MLpy came as a simple question <i>and</i> a big project in pespective:
<br><b>"Can I build a discord bot that can tell two pictures apart?"</b></p>
<br>The goal of this notebook is two-fold with one overarching thread:

1. To build a web crawler that can lift a statistically relevant number of images from [derpibooru](https://derpibooru.org), an image database powered by the community that built around the fourth generation of the show 'My Little Pony.'
2. To build a machine learning algorithm capable of telling the difference between 2 types of pictures--to be summarized in a function that I can feed to my existing discord bot [BotJack](https://github.com/LMquentinLR/botjack_discord_bot).

The thread is that I am, at the time of writing, learning how to program. I neither know how to build a web crawler or how a ML algorithm works (is it even called an algorithm?). All in all, this is a small idea that is both a learning experience, a blog--and of course a fun project.

### Why a bot should do that?
There are many reasons why a bot should be able to identify images posted on a server: classification, tagging, games, etc. 
<p style="text-align: center;"><br>This notebook will focus on <b>compliance</b>.</p> 

* Servers may have anti-NSFW (i.e. not safe for watch) rules where explicit, grim, and otherwise unwanted content is banned or curtailed to specific server channels.
* Moderation being volunteer-driven on discord, malicious users may capitalize on idle, asleep, or away-from-keyboard moderators to engage in rule-breaking activities. More commonly, users may simply post a NSFW picture in a SFW-only channel. 
* A bot able to distinguish NSFW content from SFW helps fill in the breaches that may affect any moderation effort. A bot, for instance, could automatically alert moderators when a specific content is posted and start a moderating process prior to any human intervention.

<b>Automatic content moderation and compliance is a current industry effort in social media (e.g. Facebook)</b>, making this notebook a real world application.

In [ ]:
import os
import json
import requests
import time
import operator

### Building a web crawler
Derpibooru is a website dedicated to fanart of MLP:FiM. It provides a JSON REST API for major site functionality, which can be freely used by anyone wanting to produce tools for the site or other webapps that use the data provided within Derpibooru.
<br><b>Derpibooru licensing rules</b>
<br>"<i>Anyone can use it, users making abusively high numbers of requests may be asked to stop. Your application MUST properly cache, respect server-side cache expiry times. Your client MUST gracefully back off if requests fail (eg non-200 HTTP code), preferably exponentially or fatally.</i>"

<br>A single image can be accessed through the following links:
1. https://derpibooru.org/2072316 (embedded)
2. https://derpicdn.net/img/view/2019/6/22/2072316.png (default size)
3. https://derpicdn.net/img/view/2019/6/22/2072316_small.png (small size)
4. https://derpicdn.net/img/view/2019/6/22/2072316_medium.png (medium size)
5. https://derpicdn.net/img/view/2019/6/22/2072316_large.png (large size)

The metadata of a single picture can be accessed through the following link:
* https://derpibooru.org/2072316.json
<br> The list of attributes a single image is:
>id, created_at, updated_at, first_seen_at, score, comment_count, width, height, file_name, description, uploader, uploader_id, image, upvotes, downvotes, faves, tags, tag_ids, aspect_ratio, original_format, mime_type, sha512_hash, orig_sha512_hash, source_url, representations, is_rendered, is_optimized, interactions, spoilered

In [ ]:
class img_metadata:
    """
    Class object that corresponds to the process retrieving picture metadata from the REST API
    of the website derpibooru--data is retrieved as a series of est. 1Mb JSON files.
    """
    
    def __init__(self, tags = "", at_least_one = True, instances = 10):
        """
        Initializes of the img_metadata class object.
        ---
        :param <self>:         <class>   ; class object reference
        :param <tags>:         <list>    ; list of strings (i.e. picture tags)
        :param <at_least_one>: <boolean> ; toggles between contains 'at least one tag' and 'all tags' in <tags>
        :param <instances>:    <integer> ; number of instances/loops allowed before program stops
        """
        self.tags = tags
        self.at_least_one = at_least_one
        self.instances = instances
    
    def convert_bytes(self, bytes_size):
        """
        Converts byte lengths.
        ---
        :param <self>:       <class>   ; class object reference
        :param <bytes_size>: <integer> ; size in bytes of a file
        """
        for unit_multiple in ['bytes', 'KB', 'MB', 'GB', 'TB']:
            if bytes_size < 1024.0:
                return "%3.1f %s" % (bytes_size, unit_multiple)
            bytes_size /= 1024.0
    
    def keys_to_keep(self):
        """
        Returns the keys to keep in the JSON extract.
        ---
        :param <self>: <class> ; class object reference
        """
        keys = ["id", "created_at", "updated_at", "score", "uploader",
                "uploader_id", "upvotes", "downvotes", "faves", "tags",
                "tags_id", "representations"]
        return keys
    
    def json_split(self, path):
        """
        Splits a json file if it is too large (1Mb).
        ---
        :param <self>: <class>  ; class object reference
        :param <path>: <string> ; path of a json file
        """
        length = self.convert_bytes(float(os.stat(path).st_size))
        length = length.split(" ")
        if float(length[0]) >= 1.0 and length[1] == "MB":
            nb_file = 0
            while True:
                new_path = path[:-5] + "_" + str(nb_file) + ".json"
                if os.path.exists(new_path) == False:
                    print("SPLIT: JSON file to be split as 1Mb max size reached.")
                    os.rename(path, new_path) 
                    break
                nb_file += 1
    
    def check_prior_extract(self, print_msg = True):
        """
        Checks existing metadata extractions in the working directory. 
        The default file name is 'derpibooru_metadata.json'.
        ---
        :param <self>:      <class>   ; class object reference
        :param <print_msg>: <boolean> ; toggle between 'prints message to command line' and 'prints nothing'
        """
        
        json_found = "FOUND: 'derpibooru_metadata.json'"
        json_not_found = f"MISSING FILE: 'derpibooru_metadata.json'; NOT IN: {os.getcwd()}\n" + \
        "FILE TO CREATE: 'derpibooru_metadata.json'"
        json_created = "FILE CREATED: 'derpibooru_metadata.json'"
        json_not_created = "ERROR FILE CREATION: 'derpibooru_metadata.json'"
        json_path = os.getcwd() + "\\derpibooru_metadata.json"
        
        find = os.path.exists(json_path)
        
        #if TRUE: opens file and extracts the contained metadata
        #if FALSE: creates file storing an empty list
        if find:
            if print_msg: print(json_found)
        else:
            if print_msg: print(json_not_found)
            try:    
                with open(json_path, "w") as file: file.write("[]")   
                print(json_created) 
            except Exception as e: print(json_not_created, e, sep = "\n")
        
        return json_path

    def json_collect(self, json_local, json_derpibooru, json_path):
        """
        Collects picture metadata extracted from derpibooru.
        ---
        :param <self>:            <class>       ; class object reference
        :param <json_local>:      <json_object> ; JSON data stored locally
        :param <json_derpibooru>: <json_object> ; JSON data extracted from derpibooru
        :param <json_path>:       <string>      ; path of local file where the data is stored
        """
        stored_keys = self.keys_to_keep()
        last_id = -1
        
        for image_data in json_derpibooru:
            
            temp = image_data.copy()
            
            for item in image_data: 
                if item not in stored_keys: del temp[item]
            
            last_id = max(image_data["id"], last_id)
            
            json_local.append(temp)
            json_local.sort(key=operator.itemgetter("id"), reverse = True)

        with open(json_path,'w') as file: json.dump(json_local, file)
        
        self.json_split(json_path)
        
        return last_id

    def crawl_metadata(self):
        """
        Retrieves from the derpibooru REST API a list of picture metadata.
        ---
        :param <self>: <class> ; class object reference
        """
        #initializes local variables    
        iterations = self.instances
        back_off_counter = 1
        max_instances_reached = "The set maximum number of images to request was reached " + \
                                f"at {self.instances}."
        exit_condition = "The crawler scraped the derpibooru metadata. The program will " + \
                         "now close."
        
        #retrieves most recent recorded picture id
        if os.path.exists("derpibooru_metadata.json"): json_path = "derpibooru_metadata.json"
        else: json_path = self.check_prior_extract()
        
        with open(json_path, "r") as file: requested_id = json.load(file)
        
        if requested_id == []: requested_id = 1
        else: requested_id = requested_id[0]["id"] + 1
        
        while True:
            requested_page = "You are requesting the derpibooru page starting with the " + \
                             f"id {requested_id}."
            error_json_extraction = "The program couldn't extract the page and " + \
                                    "will now proceed to an exponential back off."
            
            #checks if previous JSON was not renamed due to the 1Mb splitting
            json_path = self.check_prior_extract(False)
            
            with open(json_path,'r') as file: json_local = json.load(file)
            
            print(requested_page)
            
            path_derpibooru = "https://derpibooru.org/images.json?constraint=id&order=a&gt=" + \
                              str(requested_id)
            
            try:
                if type(iterations) == int:
                    if iterations > 1: iterations -= 1
                    else: break
                
                json_derpibooru = requests.get(path_derpibooru).json()["images"]
                if json_derpibooru == []: raise DatabaseFullyCrawled
                
                requested_id = self.json_collect(json_local, json_derpibooru, json_path)
                
                #time delay to respect the API's license
                time.sleep(.250)
            
            except DatabaseFullyCrawled:
                print(exit_condition)
                break
            
            except Exception as e:
                print(e)
                print(error_json_extraction)
                print(f"The error was the following: {e}.\n The program will back " + \
                      f"off for {2**back_off_counter} seconds.")
                back_off_counter += 1
                time.sleep(2 ** back_off_counter)
    
    def id_filter(self, tags, at_least_one):
        """
        Retrieves the IDs of the locally stored metadata that fit specific tag parameters
        ---
        :param <self>:         <class>   ; class object reference
        :param <tags>:         <list>    ; list of strings (i.e. picture tags)
        :param <at_least_one>: <boolean> ; toggles between contains 'at least one tag' and 'all tags' in <tags>
        """
        def any_or_all(boolean):
            """
            Returns the function any() if argument <boolean> TRUE, else returns function all()
            ---
            :param <boolean>: <boolean> ; boolean value
            """
            if boolean == True:
                return any
            else:
                return all
        
        metadata_files_list = filter(lambda file: file.startswith("derpibooru_metadata"), 
                                     os.listdir(os.getcwd()))
        metadata_files_list = list(metadata_files_list)
        
        id_list = []
        url_list = []
        
        for fname in metadata_files_list:
            
            with open(fname,"r") as file: json_local = json.load(file)
            if json_local == []: break
            
            tags_keep = list(filter(lambda item: item.startswith("+"), tags))
            tags_keep = list(map(lambda item: item[1:], tags_keep))
            tags_remove = list(filter(lambda item: item.startswith("-"), tags))
            tags_remove = list(map(lambda item: item[1:], tags_remove))
            
            fltr = any_or_all(at_least_one)
            filter_keep = lambda item: fltr(tag in item["tags"].split(", ") for tag in tags_keep)
            filter_remove = lambda item: not any(tag in item["tags"].split(", ") for tag in tags_remove)
            json_kept = list(filter(filter_keep, json_local))
            json_kept = list(filter(filter_remove, json_kept))
            
            filter_id = lambda item: item["id"]
            filter_url = lambda item: item["representations"]["tall"][2:]
            id_list += list(map(filter_id, json_kept))
            url_list += list(map(filter_url, json_kept))
            
        return list(zip(id_list, url_list))
    
    def repair_tags(self):
        """
        Checks if all retrieved IDs have an available list of tags.
        ---
        :param <self>:         <class>    ; class object reference
        """
        metadata_files_list = filter(lambda file: file.startswith("derpibooru_metadata"), 
                                     os.listdir(os.getcwd()))
        metadata_files_list = list(metadata_files_list)
        
        for fname in metadata_files_list:
            
            with open(fname, "r") as file: 
                
                json_local = json.load(file)
                if json_local == []:break
                
                for index, item in enumerate(json_local):

                    if item["tags"] == None:
                        json_id = item["id"]
                        path_derpibooru = "https://derpibooru.org/" + str(json_id) + ".json"

                        try:
                            json_derpibooru = requests.get(path_derpibooru).json()
                            time.sleep(.250)
                            if json_derpibooru["tags"] == None: raise AbsentTagList
                            json_local[index]["tags"] = json_derpibooru["tags"]
                            print(f"The tags of the picture {json_id} were updated.")

                        except AbsentTagList:
                            print(f"The url request for the picture {json_id} returned an empty list of tags.")

                        except Exception as e:
                            print(e)
                
            with open(fname,'w') as file: json.dump(json_local, file)

In [ ]:
class derpibooru_search(img_metadata):
    """
    Class object that corresponds to a search prompting the derpibooru REST API and
    retrieve both picture metadata and the affiliated pictures.
    """
    
    def change_search(self, tags = "", at_least_one = True, instances = 50):
        """
        Changes the arguments of the created object derpibooru_search.
        ---
        :param <self>:         <class>    ; class object reference
        :param <tags>:         <list>     ; list of strings (i.e. picture tags) used for sorting
        :param <tags>:         <list>    ; list of strings (i.e. picture tags)
        :param <at_least_one>: <boolean> ; toggles between contains 'at least one tag' and 'all tags' in <tags>
        """
        self.tags = tags
        self.at_least_one = at_least_one
        self.instances = instances

    def crawl(self):
        """
        Changes the arguments of the created object derpibooru_search.
        ---
        :param <self>: <class> ; class object reference
        """
        print("----|Entering Derpibooru Data Crawler code|----")
        self.crawl_metadata()
        print("---------------|Exiting Program|---------------")
    
    def retrieve_ids(self):
        """
        Retrieves the IDs of the locally stored metadata that fit specific tag parameters.
        ---
        :param <self>: <class> ; class object reference
        """
        print("----|Retrieving IDs based on tag selection|----")
        id_list = self.id_filter(self.tags, self.at_least_one)
        print("----------------|IDs retrieved|----------------")
        return id_list
    
    def repair(self):
        """
        Repairs missing tags of the locally stored metadata .
        ---
        :param <self>: <class> ; class object reference
        """
        print("----|Repairing missing tags in stored JSON|----")
        self.repair_tags()
        print("----------------|Tags repaired|----------------")

In [ ]:
class Error(Exception):
    """Base class for other exceptions"""
    pass

class DatabaseFullyCrawled(Error):
    """Raised when the crawler reached the last pages of derpibooru"""
    pass

class NewContentCrawled(Error):
    """Raised when the input value is too large"""
    pass

class AbsentTagList(Error):
    """Raised when the key value of the key 'tags' in a dictionary is 'None'"""
    pass

------------------

## Initializing object

In [ ]:
obj = derpibooru_search()

## Crawling Tests
1. Test #1 would retrieve all existing available picture metadata but will stop at the 10th requested page
2. Test #2 will retrieve all existing available picture metadata (can take up to 60h as at July 2019)

In [ ]:
obj.__dict__
obj.change_search(instances = 10)
obj.crawl()

In [ ]:
obj.__dict__
obj.change_search(instances = "")
obj.crawl()

Sometimes, retrieved metadata would be missing their tags. The following repairs them.

In [ ]:
obj.repair()

## Retrieving IDs based on tag selection

This version of the search function takes a list of strings which must match the following format:
<br><br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"+tag" <i>or</i> "-tag"
<br><br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<b>+</b> indicates keeping the ID for search, <b>-</b> indicates removing the ID for search.
<br><br>The [at_least_one] variable indicates:
1. if TRUE: the retrieved IDs will <b>only</b> be the ones which tags <b>contain at least one</b> of the items listed in the variable [tags] with a "+" prefix
2. if FALSE: the retrieved IDs will <b>only</b> be the ones which tags <b>contain all</b> of the items listed in the variable [tags]

In [ ]:
search_tags = ["+rarity","+applejack","+ship", "-animated"]
search_include = False
obj.change_search(tags = search_tags, at_least_one = search_include)
id_list = obj.retrieve_ids()

print(len(id_list))
print(id_list)

## Retrieving pictures based on retrieved IDs